In [2]:
import sys
sys.path.insert(1, '../../models')
from CoreSNN import save_obj, load_obj
sys.path.insert(1, '../')
from DatasetBuilding import generate_dataset
sys.path.insert(1, '../../experiments')
from ExplanationEvaluation import sample_n_testset
import pandas as pd
import numpy as np

In [2]:
# size of the synthetic dataset: 900*1000, 70-30 split between test and train
syn_ds_size = 900*1000

In [3]:
np.random.seed(123)
X_syn = np.array([[0,0]])

while len(X_syn) < syn_ds_size:
    random_duration = np.random.randint(600)
    next_activity = np.random.randint(0,2, (1,2)).repeat(random_duration, axis=0)
    X_syn = np.append(X_syn, next_activity, 0)

In [4]:
df_data_syn = pd.DataFrame(X_syn[:syn_ds_size], columns=['x1', 'x2'])

In [5]:
# add constantly spiking bias
df_data_syn.insert(loc=0, column='x0', value = np.ones(syn_ds_size, dtype=np.int8))

In [6]:
def assign_label(x1, x2):
    """
    Class 0 if x1 on, x2 off
    Class 1 if both on
    Class 2 if x1 off, x2 on
    Class 3 if both off
    """
    if (x1, x2)==(0,1):
        return 2
    elif (x1, x2)== (0,0):
        return 3
    else:
        return (x1*x2)

In [7]:
df_data_syn['Class'] = df_data_syn.apply(lambda x: assign_label(x[1], x[2]), axis=1)
df_data_syn['set'] = ['train']*630000 + ['test']*270000

In [9]:
df_data_syn.to_csv('syn_data.csv', index = False)
np.random.seed(123)
expl_testset = sample_n_testset(df_data_syn[df_data_syn['set']=='test']['Class'], 25)
save_obj(expl_testset, 'expl_syn_testset.pkl')

In [11]:
# Generate the dataset for training the model

X_train, y_train = generate_dataset(df_data_syn[df_data_syn['set']=='train'], 900)
X_test, y_test = generate_dataset(df_data_syn[df_data_syn['set']=='test'], 900)

X_train['times'] = X_train['times'][:-1]
X_train['units'] = X_train['units'][:-1]
X_test['times'] = X_test['times'][:-1]
X_test['units'] = X_test['units'][:-1]
y_train = y_train[:-1]
y_test = y_test[:-1]

dataset_syn = {'X_train': X_train,
               'y_train': y_train,
               'X_test': X_test,
               'y_test': y_test}

save_obj(dataset_syn, 'syn_data900.pkl')


<ipython-input-10-8d3b261cedfd>:36: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  data['times'] = np.array(data['times'])
<ipython-input-10-8d3b261cedfd>:37: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  data['units'] = np.array(data['units'])


In [16]:
# Generate the dataset with "long" time series

X_train, y_train = generate_dataset(df_data_syn[df_data_syn['set']=='train'], len(df_data_syn[df_data_syn['set']=='train']))
X_test, y_test = generate_dataset(df_data_syn[df_data_syn['set']=='test'], len(df_data_syn[df_data_syn['set']=='test']))

dataset_syn = {'X_train': X_train,
               'y_train': y_train,
               'X_test': X_test,
               'y_test': y_test}

save_obj(dataset_syn, 'syn_data.pkl')

